In [ ]:
!pip install -U langchain-openai langchain google-search-results wikipedia sqlalchemy
OPENAI_API_KEY= "YOUR_API_KEY"

In [16]:
from langchain_openai import OpenAI

# initialize the models
llm = OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=OPENAI_API_KEY
)

Créer une base de données

In [17]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [18]:
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)

In [19]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [20]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [21]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [22]:
for obs in observations:
    insert_obs(obs)

In [23]:
from langchain_community.utilities import SQLDatabase
db = SQLDatabase(engine)

## Types d'agents

Dans cette section, nous examinerons plusieurs agents et verrons comment ils "pensent" et ce qu'ils peuvent faire.

L'utilisation d'un des agents pré-construits de langchain implique trois variables :
* définir les outils ou la boîte à outils
* définir l'llm
* définir le type d'agent

Tout cela est assez facile à faire dans langchain, comme nous le verrons dans l'exemple suivant:

In [24]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=3
)